In [1]:
!pip install -q pinecone protoc_gen_openapiv2 grpcio

In [2]:
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

In [3]:

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key="a528878d-5605-4fa2-8b32-2aad80b82a04")


In [4]:

# Define a sample dataset where each item has a unique ID and piece of text
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
    {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
]


In [5]:
# Convert the text into numerical vectors that Pinecone can index
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)

print(embeddings)


EmbeddingsList(
  model='multilingual-e5-large',
  vector_type='dense',
  data=[
    {'vector_type': dense, 'values': [0.04931640625, -0.01328277587890625, ..., -0.0196380615234375, -0.010955810546875]},
    {'vector_type': dense, 'values': [0.032562255859375, -0.027862548828125, ..., -0.0200653076171875, -0.021026611328125]},
    ... (2 more embeddings) ...,
    {'vector_type': dense, 'values': [0.0312347412109375, -0.0185699462890625, ..., -0.02996826171875, -0.032989501953125]},
    {'vector_type': dense, 'values': [0.03955078125, -0.01013946533203125, ..., 0.0011348724365234375, -0.04296875]}
  ],
  usage={'total_tokens': 130}
)


In [6]:
# Create a serverless index
index_name = "example-index"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )



In [7]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)


In [8]:
# Upsert vectors
# Target the index where you'll store the vector embeddings
index = pc.Index(index_name)

# Prepare the records for upsert
# Each contains an 'id', the embedding 'values', and the original text as 'metadata'
records = []
for d, e in zip(data, embeddings):
    records.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

In [9]:
records[0]

{'id': 'vec1',
 'values': [0.04931640625,
  -0.01328277587890625,
  -0.01119232177734375,
  -0.038421630859375,
  0.041290283203125,
  -0.053253173828125,
  -0.018524169921875,
  0.05535888671875,
  0.03753662109375,
  -0.04974365234375,
  0.04010009765625,
  0.01464080810546875,
  -0.051544189453125,
  0.0009832382202148438,
  0.0018377304077148438,
  0.00783538818359375,
  -0.0268707275390625,
  0.018096923828125,
  -0.006473541259765625,
  -0.030853271484375,
  0.013671875,
  -0.0186614990234375,
  -0.042327880859375,
  -0.0460205078125,
  -0.047210693359375,
  -0.01424407958984375,
  -0.048828125,
  -0.0253448486328125,
  -0.0152435302734375,
  -0.045196533203125,
  0.0063934326171875,
  0.0185394287109375,
  -0.0384521484375,
  -0.027252197265625,
  -0.014801025390625,
  0.037994384765625,
  0.0197601318359375,
  -0.009490966796875,
  -0.0159912109375,
  -0.027587890625,
  -0.0259552001953125,
  0.058807373046875,
  -0.0201568603515625,
  -0.0260467529296875,
  -0.0194854736328125

In [10]:
# Upsert the records into the index
index.upsert(
    vectors=records,
    namespace="example-namespace-sandeep"
)

upserted_count: 6

In [20]:
#time.sleep(10)  # Wait for the upserted vectors to be indexed

print(index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'example-namespace-Jayanth': {'vector_count': 7},
                'example-namespace-Rahul': {'vector_count': 6},
                'example-namespace-VenkatM': {'vector_count': 6},
                'example-namespace-a': {'vector_count': 7},
                'example-namespace-ank': {'vector_count': 7},
                'example-namespace-ar': {'vector_count': 7},
                'example-namespace-gowtham': {'vector_count': 6},
                'example-namespace-pradeepN': {'vector_count': 6},
                'example-namespace-ramesh': {'vector_count': 8},
                'example-namespace-rohini': {'vector_count': 6},
                'example-namespace-sandeep': {'vector_count': 6},
                'example-namespace-sohail': {'vector_count': 6},
                'ganesh': {'vector_count': 6},
                'mehak': {'vector_count': 7}},
 'total_vector_count': 91}


In [17]:
# Define your query
#query = "Tell me about the tech company known as Apple."
query = "whicf fruit is good for health."

In [18]:
# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

In [19]:
# Search the index for the three most similar vectors
results = index.query(
    namespace="example-namespace-sandeep",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec1',
              'metadata': {'text': 'Apple is a popular fruit known for its '
                                   'sweetness and crisp texture.'},
              'score': 0.82731557,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec5',
              'metadata': {'text': 'An apple a day keeps the doctor away, as '
                                   'the saying goes.'},
              'score': 0.8212011,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec3',
              'metadata': {'text': 'Many people enjoy eating apples as a '
                                   'healthy snack.'},
              'score': 0.8084524,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': 'example-namespace-sandeep',
 'usage': {'read_units': 1}}
